In [ ]:
%cd ~/Dev/video-membership

In [ ]:
from app import db
from app.users.models import User
from cassandra.cqlengine.management import sync_table

In [ ]:
db.get_session()
sync_table(User)

In [ ]:
post_data = {
    "email": "abc@abc.com",
    "password": "abc123"
}

In [ ]:
from pydantic import BaseModel, EmailStr, SecretStr, validator

In [ ]:
class UserLoginSchema(BaseModel):
    email: EmailStr
    password: SecretStr

In [ ]:
UserLoginSchema(**post_data) # email="abc@abc", password="abc123"

In [ ]:
class UserSignupSchema(BaseModel):
    email: EmailStr
    password: SecretStr
    password_confirm: SecretStr
        
    
    @validator("email")
    def email_available(cls, v, values, **kwargs):
        q = User.objects.filter(email=v)
        if q.count() != 0:
            raise ValueError("Email is not available")
        return v
        
    @validator("password_confirm")
    def passwords_match(cls, v, values, **kwargs):
        password = values.get('password')
        password_confirm = v
        if password != password_confirm:
            raise ValueError("Passwords do not match")
        return v

In [ ]:
data = UserSignupSchema(email='abc@teamcfe.com', password='abc123', password_confirm='abc123')

In [ ]:
data.dict()['password'].get_secret_value()